In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [2]:
import json
import geojson
import pathlib
from spacetx_spatial_analysis import depth_coordinate
import boto3


## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load spot table from 1 dataset

In [4]:
s3_resource = boto3.resource('s3')

In [22]:
cwd = pathlib.Path.cwd()

In [25]:
localdir = cwd.joinpath("local_data")

In [26]:
spot_fname = "spacejam2/BARISTASEQ/all_spots_2.csv"
local_file = localdir.joinpath(spot_fname)
local_file.parent.mkdir( parents = True, exist_ok = True)
print(local_file.parent)

/Users/brian/local_data/spacejam2/BARISTASEQ


In [27]:
s3_resource.Object("starfish.data.spacetx", spot_fname).download_file(str(local_file))

In [54]:
# also download the cell x gene table for later
cell_fname = "spacejam2/BARISTASEQ/cellxgene2.csv"
local_cell_file = localdir.joinpath(cell_fname)
local_cell_file.parent.mkdir( parents = True, exist_ok = True)
print(local_cell_file.parent)
s3_resource.Object("starfish.data.spacetx", cell_fname).download_file(str(local_cell_file))

/Users/brian/local_data/spacejam2/BARISTASEQ


In [28]:
# load one spotxgene table
all_spots = pd.read_csv(local_file)
all_spots

,Unnamed: 0,Gene ID,Cell ID,x,y,x(tile),y(tile),Dist to pia,Dist to bottom,angle,target
0,0,52,10027,1172.440,1158.560,103,310,810.111,448.925,172.287,Npy
1,1,17,10087,1172.440,1250.370,103,875,757.449,514.318,166.784,Deptor
2,2,61,10127,1172.440,1334.550,103,1393,716.143,575.620,157.153,Gm11549
3,3,17,10127,1172.440,1335.040,103,1396,715.925,575.990,157.087,Deptor
4,4,28,10127,1172.440,1337.800,103,1413,714.699,578.090,156.710,Adarb2
...,...,...,...,...,...,...,...,...,...,...,...
847047,847047,61,750192,308.262,329.225,1897,1722,477.726,724.373,157.483,Gm11549
847048,847048,42,750000,309.075,355.875,1902,1886,461.572,731.747,160.071,Nrn1
847049,847049,72,750142,311.838,264.713,1919,1325,524.293,701.246,152.354,Rprml
847050,847050,23,750000,314.600,176.150,1936,780,592.405,672.597,143.313,Gad1


In [30]:
all_spots["gene"] = all_spots["target"]
genes = list(all_spots["gene"].unique())

In [31]:
subsample = 1 # change to >1 if number of points is too high
colorlist = ["magenta", "red","blue","cyan", "green", "orange","red"]


In [33]:
cindex = 0
for ii, gene_to_plot in enumerate(["Gad1","Gad2","Slc17a7","Cux2","Calb1","Meis2","Foxp2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in genes:
        if gene_to_plot == "Slc17a7":
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot][["x","y"]].values[::subsample,:], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 1.5,
                                 face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.8)
        else:
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot][["x","y"]].values[::subsample,:], symbol='disc',
                     name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 2.,
                     face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.7)
        cindex = cindex + 1

In [34]:
v.layers

[<Points layer 'Gad1+' at 0x12b66cc90>,
 <Points layer 'Gad2+' at 0x12b739450>,
 <Points layer 'Slc17a7+' at 0x122d867d0>,
 <Points layer 'Calb1+' at 0x1291b3bd0>,
 <Points layer 'Atp1a3+' at 0x1200d8c50>,
 <Points layer 'Vxn+' at 0x12023b0d0>,
 <Points layer 'Pcp4+' at 0x129fe7650>,
 <Points layer 'Fezf2+' at 0x12a55fcd0>,
 <Points layer 'Rorb+' at 0x12a565290>,
 <Points layer 'Gm11549+' at 0x12cd14f90>,
 <Shapes layer 'Shapes' at 0x12e611990>]

### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire area
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. white matter
6. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [35]:
shapes_layer = v.layers[-1].data[:-1]
column_points = v.layers[-1].data[-1]

In [37]:
if len(v.layers[-1].data) != 8:
    print("please annotate the 7 polygons and 1 line segment described above")
else:
    print(str(len(v.layers[-1].data))+" annotations")

8 annotations


In [39]:
# go get all the plotted points and note if they're in the annotated polygons
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in all_spots[["x","y"]].values] for ii,p in enumerate(shapes_layer)}

In [41]:
annotation_names = ["VISp", "VISp_I", "VISp_II/III", "VISp_IV", "VISp_V","VISp_VI", "VISp_wm", "column_axis"]


In [42]:
all_spots["layer"] = "outside_VISp"
for k in is_in_layer.keys():
    all_spots.loc[is_in_layer[k],["layer"]] = annotation_names[k]


In [ ]:
# now the spots have depth coordinates, which will be useful for detected spots v. column depth
all_spots["depth_um"] = depth_coordinate(all_spots[["x","y"]].values, column_points)

In [46]:
all_spots

,Unnamed: 0,Gene ID,Cell ID,x,y,x(tile),y(tile),Dist to pia,Dist to bottom,angle,target,gene,layer
0,0,52,10027,1172.440,1158.560,103,310,810.111,448.925,172.287,Npy,Npy,outside_VISp
1,1,17,10087,1172.440,1250.370,103,875,757.449,514.318,166.784,Deptor,Deptor,outside_VISp
2,2,61,10127,1172.440,1334.550,103,1393,716.143,575.620,157.153,Gm11549,Gm11549,outside_VISp
3,3,17,10127,1172.440,1335.040,103,1396,715.925,575.990,157.087,Deptor,Deptor,outside_VISp
4,4,28,10127,1172.440,1337.800,103,1413,714.699,578.090,156.710,Adarb2,Adarb2,outside_VISp
...,...,...,...,...,...,...,...,...,...,...,...,...,...
847047,847047,61,750192,308.262,329.225,1897,1722,477.726,724.373,157.483,Gm11549,Gm11549,outside_VISp
847048,847048,42,750000,309.075,355.875,1902,1886,461.572,731.747,160.071,Nrn1,Nrn1,outside_VISp
847049,847049,72,750142,311.838,264.713,1919,1325,524.293,701.246,152.354,Rprml,Rprml,outside_VISp
847050,847050,23,750000,314.600,176.150,1936,780,592.405,672.597,143.313,Gad1,Gad1,outside_VISp


In [48]:
all_spots.to_csv(local_file.parent.joinpath("all_spots_2_with_annotation.csv"))

In [50]:
annotations = [mapping(Polygon(p))  for ii,p in enumerate(shapes_layer)]
annotations.append(mapping(LineString(column_points)))
for ii,ai in enumerate(annotations):
    ai["name"] = annotation_names[ii]

In [51]:
a = geojson.GeometryCollection(annotations)

In [53]:
with open(local_file.parent.joinpath("annotations_geo.json"),'w') as w:
    geojson.dump(a,w)

In [ ]:
with open("/Users/brian/work/data/stardist_rois.json",'r') as r:
    shapejson = geojson.load(r)

In [57]:
all_cells = pd.read_csv(local_cell_file)

In [61]:
cells = v.add_points(all_cells[["x_um","y_um"]], symbol='o', name = "all", opacity = 0.8, blending = "translucent")
colorlist = ["magenta", "green","yellow","cyan", "blue", "orange"]
for ii, gene_to_plot in enumerate(["Cux2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in all_cells:
        cux2condition = all_cells[gene_to_plot]>0.5*all_cells[gene_to_plot].median()
        cux2cells = v.add_points(all_cells[cux2condition][["x_um","y_um"]], symbol='disc',
                                 name = gene_to_plot+"+ cells",edge_color= "black", size = 5,
                                 face_color= colorlist[ ii % len(colorlist)] , blending = "translucent", opacity = 0.3)
    

In [64]:
cell_is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in all_cells[["x_um","y_um"]].values] for ii,p in enumerate(shapes_layer)}

In [67]:
all_cells["depth_um"] = depth_coordinate(all_cells[["x_um","y_um"]].values, column_points)

In [68]:
all_cells["layer"] = "outside_VISp"
for k in cell_is_in_layer.keys():
    all_cells.loc[cell_is_in_layer[k],["layer"]] = annotation_names[k]


In [69]:
all_cells["layer"].unique()

array(['outside_VISp', 'VISp_VI', 'VISp_wm', 'VISp', 'VISp_V', 'VISp_IV',
       'VISp_II/III', 'VISp_I'], dtype=object)

In [70]:
all_cells.to_csv(local_cell_file.parent.joinpath("cellxgene2_with_depth.csv"))